In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV

# from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [5]:
df = pd.read_csv('stud.csv')

In [6]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [7]:
X= df.drop(columns =['math_score'],axis=1)
y = df['math_score']

In [8]:
numerical_features = [feature for feature in X.columns if df[feature].dtype !='O']
categorical_features = [feature for feature in X.columns if df[feature].dtype=='O']

print(numerical_features)
print(categorical_features)

['reading_score', 'writing_score']
['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch', 'test_preparation_course']


In [16]:
# convert cat features into onehotendoing, once done apply standarization on all columns
numerical_features = [feature for feature in X.columns if df[feature].dtype !='O']
categorical_features = [feature for feature in X.columns if df[feature].dtype=='O']

numerical_transformer = StandardScaler()
oh_transfomer = OneHotEncoder()

preprocessor = ColumnTransformer(

    [
        ("OneHotEncoder",oh_transfomer, categorical_features),
        ("StandardScaler",numerical_transformer,  numerical_features)

    ]
)

In [18]:
X= preprocessor.fit_transform(X)

In [20]:
X.shape

(1000, 19)

In [23]:
X_train, X_test , y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

In [24]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_squared = r2_score(true, predicted)
    return mae, rmse, r2_squared


In [34]:
models ={
            "Linear regression": LinearRegression(),
            "Lasso": Lasso(),
            "Ridge": Ridge(),
            "Decision Tree": DecisionTreeRegressor(),
            "Random forest" : RandomForestRegressor(),
            "XGB Regressor": XGBRegressor(),
            "Ada boost Regressor" : AdaBoostRegressor()

}

model_list =[]
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.values())[i])
    model_list.append(list(models.keys())[i])

    print("Model performance for training set")
    print("- Root mean squared error: "+ str(model_train_rmse))
    print("- Mean absolute error: "+ str(model_train_mae))
    print("- R2 Score: "+ str(model_train_r2))

    print('-------------------------------------------')

    print("Model performance for testing set")
    print("- Root mean squared error: "+ str(model_test_rmse))
    print("- Mean absolute error: "+ str(model_test_mae))
    print("- R2 Score: "+ str(model_test_r2))

    r2_list.append(model_test_r2)

    print('='*35)
    print('\n')
    
    
    

LinearRegression()
Model performance for training set
- Root mean squared error: 5.32433481852575
- Mean absolute error: 4.267109375
- R2 Score: 0.8742565651513869
-------------------------------------------
Model performance for testing set
- Root mean squared error: 5.3959872842671395
- Mean absolute error: 4.2158203125
- R2 Score: 0.8803449074540941


Lasso()
Model performance for training set
- Root mean squared error: 6.593815587795566
- Mean absolute error: 5.206302661246526
- R2 Score: 0.8071462015863456
-------------------------------------------
Model performance for testing set
- Root mean squared error: 6.519694535667419
- Mean absolute error: 5.157881810347763
- R2 Score: 0.8253197323627853


Ridge()
Model performance for training set
- Root mean squared error: 5.323324922741654
- Mean absolute error: 4.264987823725981
- R2 Score: 0.8743042615212909
-------------------------------------------
Model performance for testing set
- Root mean squared error: 5.390387016935642
- M

In [35]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
2,Ridge,0.880593
0,Linear regression,0.880345
4,Random forest,0.852925
6,Ada boost Regressor,0.850947
5,XGB Regressor,0.827797
1,Lasso,0.825320
3,Decision Tree,0.748868


In [36]:
lin_model = LinearRegression(fit_intercept=True)
lin_model = lin_model.fit(X_train, y_train)
y_pred = lin_model.predict(X_test)
score = r2_score(y_test, y_pred)*100
print(" Accuracy of the model is %.2f" %score)

 Accuracy of the model is 88.03


In [37]:
pred_df=pd.DataFrame({'Actual Value':y_test,'Predicted Value':y_pred,'Difference':y_test-y_pred})
pred_df

,Actual Value,Predicted Value,Difference
521,91,76.507812,14.492188
737,53,58.953125,-5.953125
740,80,76.960938,3.039062
660,74,76.757812,-2.757812
411,84,87.539062,-3.539062
...,...,...,...
408,52,43.546875,8.453125
332,62,62.031250,-0.031250
208,74,67.976562,6.023438
613,65,67.132812,-2.132812
